In [ ]:
'''This notebook is used to process the inpatient encounters encounter dataset from the Centers for 
Medicare and Medicaid Services. The inpatient dataset only contains information from 2015-2023.'''

import pandas as pd
import numpy as np
np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
from utils import describe_dataframe

In [2]:
print(os.getcwd())

/Users/lukewilsen/Desktop/School/BS_696_ind_study/HealthCareAnalysis/luke_code/data_processing


In [4]:
inpatient = pd.read_csv("../data/inpatient_raw.csv", sep = "|")

/var/folders/1k/3mrcqp5d4670v77qsx12xwmm0000gn/T/ipykernel_47686/3204922659.py:1: DtypeWarning: Columns (168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189) have mixed types. Specify dtype option on import or set low_memory=False.
  inpatient = pd.read_csv("../data/inpatient_raw.csv", sep = "|")


In [7]:
# Create the dataframe that describes the columns of inpatient
inpatient_description = describe_dataframe(inpatient)
inpatient_description


,column_name,dtype,non_null_count,null_count,unique_count,sample_values,min,max,mean
0,BENE_ID,int64,58066,0,5699,"[-10000010254618, -10000010254653, -1000001025...",-10000010288010,-10000010254618,-10000010270388.378906
1,CLM_ID,int64,58066,0,20867,"[-10000930037831, -10000930038030, -1000093003...",-10000931486024,-10000930037831,-10000930736409.173828
2,NCH_NEAR_LINE_REC_IDENT_CD,object,58066,0,1,[V],None,None,None
3,NCH_CLM_TYPE_CD,int64,58066,0,1,[60],60,60,60.0
4,CLM_FROM_DT,object,58066,0,2914,"[25-Mar-2015, 24-Sep-2015, 09-May-2017, 14-Jan...",None,None,None
...,...,...,...,...,...,...,...,...,...
192,CLM_UNCOMPD_CARE_PMT_AMT,float64,0,58066,0,[],NaN,NaN,NaN
193,CLM_LINE_NUM,int64,58066,0,46,"[1, 2, 3, 4, 5]",1,46,3.34137
194,REV_CNTR,int64,58066,0,2,"[450, 1]",1,450,334.189147
195,HCPCS_CD,object,58066,0,106,"[99221, 99024, 73610, 29515, 95953]",None,None,None


In [ ]:
# Checks to see the years of the dataset UPON LOADING because enrollment dataset had years from 2015-2025

pd.to_datetime(inpatient['CLM_FROM_DT'], format='%d-%b-%Y').dt.year.unique()

array([2015, 2017, 2018, 2022, 2021, 2020, 2023, 2016, 2019])

In [11]:
# Import ICD code descriptions for futur merging

icd_codes = pd.read_csv("../data/ICD_10_CM_codes_2025.txt", sep = "  ", header=None)
icd_codes['Description'] = np.where(icd_codes[1].isna(), icd_codes[2], icd_codes[1])
icd_codes.drop(columns = [1,2], inplace=True)
icd_codes.columns = ['ICD_Code', 'ICD_Description']
icd_codes

/var/folders/1k/3mrcqp5d4670v77qsx12xwmm0000gn/T/ipykernel_47686/2152624378.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  icd_codes = pd.read_csv("../data/ICD_10_CM_codes_2025.txt", sep = "  ", header=None)


,ICD_Code,ICD_Description
0,A000,"Cholera due to Vibrio cholerae 01, biovar chol..."
1,A001,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,A009,"Cholera, unspecified"
3,A0100,"Typhoid fever, unspecified"
4,A0101,Typhoid meningitis
...,...,...
74260,Z9981,Dependence on supplemental oxygen
74261,Z9989,Dependence on other enabling machines and dev...
74262,U070,Vaping-related disorder
74263,U071,COVID-19


In [13]:
# Check to see examples of missing ICD_Descriptions

icd_codes[icd_codes["ICD_Description"].isna()]

,ICD_Code,ICD_Description
1309,C441021\tUnspecified malignant neoplasm of ski...,None
1310,C441022 Unspecified malignant neoplasm of skin...,None
1311,C441091 Unspecified malignant neoplasm of skin...,None
1312,C441092 Unspecified malignant neoplasm of skin...,None
1314,C441121 Basal cell carcinoma of skin of right ...,None
...,...,...
72424,"Y389X2S Terrorism, secondary effects, civilian...",None
73905,"Z860100 Personal history of colon polyps, unsp...",None
73906,Z860101 Personal history of adenomatous and se...,None
73907,Z860102 Personal history of hyperplastic colon...,None


In [14]:
# If the icd description is missing, it's because the value in the ICD_Code column is a 
# Combination of the code and the description. Therefore, the ICD_Code column needs to be split into two columns:
# ICD_Code and ICD_Description. The first part of the string is the code, and the second part is the description.

icd_codes[["ICD_Code", "ICD_Description"]] = icd_codes.apply(
    lambda row: pd.Series(row["ICD_Code"].split(' ', 1)) if pd.isna(row["ICD_Description"]) else pd.Series([row["ICD_Code"], row["ICD_Description"]]),
    axis=1
)

In [ ]:
# Check to make sure no icd descriptions are missing
icd_codes[icd_codes["ICD_Description"].isna()]

,ICD_Code,ICD_Description


In [17]:
# Load in DRG code and descriptions for future merging

drg_codes = pd.read_excel("../data/MS-DRG-1023.xlsx", header = 1)
drg_codes = drg_codes[["MS-DRG ", "MS-DRG Title"]]
drg_codes.rename(columns = {"MS-DRG ":"DRG_CODE", "MS-DRG Title":"DRG_TITLE"}, inplace=True)
drg_codes["DRG_CODE"] = drg_codes["DRG_CODE"].astype(str).str.zfill(3)
drg_codes.head()

,DRG_CODE,DRG_TITLE
0,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...
1,002,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...
2,003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...
3,004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...
4,005,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...


In [18]:
# Process specific columns from the inpatient dataset

# Convert code columns to strings
inpatient["BENE_ID"] = inpatient["BENE_ID"].astype(str)
inpatient["CLM_ID"] = inpatient["CLM_ID"].astype(str)
inpatient['PTNT_DSCHRG_STUS_CD'] = inpatient['PTNT_DSCHRG_STUS_CD'].astype(str)
inpatient['CLM_IP_ADMSN_TYPE_CD'] = inpatient['CLM_IP_ADMSN_TYPE_CD'].astype(str)

#Convert date columns to date
inpatient['CLM_FROM_DT'] = pd.to_datetime(inpatient['CLM_FROM_DT'], format='%d-%b-%Y')
inpatient['CLM_THRU_DT'] = pd.to_datetime(inpatient['CLM_THRU_DT'], format='%d-%b-%Y')

#calculate LOS and YR
inpatient['LOS'] = (inpatient['CLM_THRU_DT'] - inpatient['CLM_FROM_DT']).dt.days
inpatient['YR'] = inpatient['CLM_FROM_DT'].dt.year

#Convert claim drg to numeric
inpatient['CLM_DRG_CD'] = pd.to_numeric(inpatient['CLM_DRG_CD'], errors='coerce')
inpatient['DRG'] = inpatient['CLM_DRG_CD'].fillna(0).astype(int).astype(str).str.zfill(3)

#drop columns
#E codes correspond to injury codes so include if interested
#POA in col for present on admission (could be used to measure infections/conditions aquired in hospital)
cols_to_drop = ['CLM_DRG_CD'] + [col for col in inpatient.columns if 'POA' in col or col.endswith("UPIN") or col.startswith("ICD_DGNS_E_CD") or col.startswith("PRCDR_DT")]
inpatient = inpatient.drop(columns=cols_to_drop)

#filter rows
inpatient = inpatient[inpatient['CLM_LINE_NUM'] == 1] #amt charged per claim is the same for each claim line num

# ER flag
# Try out including ER flag with LOS > 1
inpatient["ER_flag"] = np.where((inpatient['REV_CNTR'] == 450) & (inpatient["LOS"] == 1), 1, 0)
inpatient["ER_outpatient_flag"] = np.where((inpatient['REV_CNTR'] == 450) & (inpatient["LOS"] == 0), 1, 0) #these obs technically belong in the outpatient file

In [36]:
inpatient["YR"].unique()

array([2015, 2017, 2018, 2022, 2021, 2020, 2023, 2016, 2019])

In [19]:
# Investigate the columns with high uniqueness/variability in their values

unique_cols = inpatient.nunique()
sorted_unique_cols = unique_cols.sort_values(ascending=True)

meaning_cols = list(sorted_unique_cols[(sorted_unique_cols != 0) & (sorted_unique_cols != 1)].index)

unique_ratio = unique_cols / len(inpatient)

# Display the columns with high uniqueness
print(unique_ratio.sort_values(ascending=False).head(20))

CLM_ID                            1.000000
CLM_PMT_AMT                       0.495567
CLM_TOT_CHRG_AMT                  0.495567
NCH_BENE_PTA_COINSRNC_LBLTY_AM    0.384914
BENE_ID                           0.273111
ORG_NPI_NUM                       0.234916
PRVDR_NUM                         0.233670
CLM_ADMSN_DT                      0.139646
CLM_FROM_DT                       0.139646
CLM_THRU_DT                       0.139455
NCH_BENE_DSCHRG_DT                0.139455
OP_PHYSN_NPI                      0.118033
AT_PHYSN_NPI                      0.118033
NCH_IP_TOT_DDCTN_AMT              0.113576
NCH_IP_NCVRD_CHRG_AMT             0.113576
NCH_PRMRY_PYR_CLM_PD_AMT          0.108449
NCH_BENE_IP_DDCTBL_AMT            0.033258
NCH_WKLY_PROC_DT                  0.019984
ICD_DGNS_CD2                      0.011597
ICD_DGNS_CD3                      0.011406
dtype: float64


In [20]:
select_vars = ["BENE_ID",
            "CLM_ID",
            "LOS",
            "CLM_PMT_AMT",
            "YR",
            "PRNCPAL_DGNS_CD",
            "DRG",
            "PTNT_DSCHRG_STUS_CD",
            "ER_flag"]

In [ ]:
# Inpatient diagnoses can consist of many different individual diagnoses. This dataframe contains all such
# diagnoses.

diagnosis = inpatient[["BENE_ID","YR","PRNCPAL_DGNS_CD"] + [col for col in inpatient.columns if col.startswith("ICD_DGNS_CD")]]


In [ ]:
#Wide to long, make sure diagnosis is not NA or empty
diagnosis = pd.melt(diagnosis,
                    id_vars = ["BENE_ID", "YR"],
                    var_name = "ICD_DIAG_COL",
                    value_name = "ICD_DIAG_CD")

diagnosis = diagnosis[diagnosis['ICD_DIAG_CD'].notna() & (diagnosis['ICD_DIAG_CD'] != "")]

diagnosis.drop(columns="ICD_DIAG_COL", inplace=True)
diagnosis = diagnosis.drop_duplicates()

In [23]:
# Example of all the diagnoses a single patient had, and in what year.
diagnosis[diagnosis["BENE_ID"] == "-10000010254691"]

,BENE_ID,YR,ICD_DIAG_CD
8,-10000010254691,2020,Z608
10,-10000010254691,2020,R5081
11,-10000010254691,2021,Z733
20877,-10000010254691,2020,P84
41742,-10000010254691,2020,Z7682
41744,-10000010254691,2020,T7432X
41745,-10000010254691,2021,R0902
62609,-10000010254691,2020,N184
62612,-10000010254691,2021,Z608
83476,-10000010254691,2020,E1121


In [24]:
# Add diagnosis description (ICD codes)

diagnosis = pd.merge(diagnosis, icd_codes, how = "left", left_on = "ICD_DIAG_CD", right_on = "ICD_Code")

In [ ]:
# Drop the ICD_Code col (duplicate of ICD_DIAG_CD)

diagnosis.drop(columns="ICD_Code", inplace=True)

In [ ]:
# Repeated example from above

diagnosis[diagnosis["BENE_ID"] == "-10000010254691"]

,BENE_ID,YR,ICD_DIAG_CD,ICD_Description
8,-10000010254691,2020,Z608,Other problems related to social environment
9,-10000010254691,2020,R5081,Fever presenting with conditions classified e...
10,-10000010254691,2021,Z733,"Stress, not elsewhere classified"
15254,-10000010254691,2020,P84,Other problems with newborn
20339,-10000010254691,2020,Z7682,Awaiting organ transplant status
20340,-10000010254691,2020,T7432X,"Child psychological abuse, confirmed"
20341,-10000010254691,2021,R0902,Hypoxemia
32840,-10000010254691,2020,N184,"Chronic kidney disease, stage 4 (severe)"
32841,-10000010254691,2021,Z608,Other problems related to social environment
45197,-10000010254691,2020,E1121,Type 2 diabetes mellitus with diabetic nephro...


In [27]:
# Calculate the number of diagnoses per year per patient

num_diag = diagnosis.groupby(['BENE_ID', 'YR']).size().reset_index(name = "NUM_DIAG")

In [ ]:
# Select specific columns to include in the inpatient dataframe, and add number of total diagnoses per year to it.

inpatient_encounters = inpatient[["BENE_ID","CLM_ID","CLM_FROM_DT",
                                  "CLM_THRU_DT","YR","LOS","DRG",
                                  'PRNCPAL_DGNS_CD','PTNT_DSCHRG_STUS_CD','CLM_IP_ADMSN_TYPE_CD',
                                  'ER_flag','CLM_TOT_CHRG_AMT','CLM_PMT_AMT']]

inpatient_encounters = inpatient_encounters.merge(num_diag, on = ["BENE_ID", "YR"], how = "left")

In [29]:
inpatient_encounters.head()

,BENE_ID,CLM_ID,CLM_FROM_DT,CLM_THRU_DT,YR,LOS,DRG,PRNCPAL_DGNS_CD,PTNT_DSCHRG_STUS_CD,CLM_IP_ADMSN_TYPE_CD,ER_flag,CLM_TOT_CHRG_AMT,CLM_PMT_AMT,NUM_DIAG
0,-10000010254618,-10000930037831,2015-03-25,2015-03-25,2015,0,551,S134XX,1,1,0,96.65,96.65,9
1,-10000010254653,-10000930038030,2015-09-24,2015-09-24,2015,0,951,Z3480,1,1,0,6311.88,6311.88,4
2,-10000010254653,-10000930038031,2017-05-09,2017-05-10,2017,1,923,T7432X,1,3,0,8545.72,8545.72,3
3,-10000010254656,-10000930038162,2017-01-14,2017-01-14,2017,0,564,S8290X,1,1,0,1014.85,1014.85,4
4,-10000010254656,-10000930038163,2018-03-17,2018-03-17,2018,0,951,Z3480,1,1,0,9911.41,9911.41,6


In [30]:
# Add icd code descriptions
inpatient_encounters = inpatient_encounters.merge(icd_codes, how = "left", left_on = "PRNCPAL_DGNS_CD", right_on = "ICD_Code")
inpatient_encounters.drop(columns="ICD_Code", inplace=True)

In [31]:
# Add DRG code descriptions
inpatient_encounters = inpatient_encounters.merge(drg_codes, how = "left", left_on = "DRG", right_on = "DRG_CODE")
inpatient_encounters.drop(columns="DRG_CODE", inplace=True)

In [32]:
inpatient_encounters

,BENE_ID,CLM_ID,CLM_FROM_DT,CLM_THRU_DT,YR,LOS,DRG,PRNCPAL_DGNS_CD,PTNT_DSCHRG_STUS_CD,CLM_IP_ADMSN_TYPE_CD,ER_flag,CLM_TOT_CHRG_AMT,CLM_PMT_AMT,NUM_DIAG,ICD_Description,DRG_TITLE
0,-10000010254618,-10000930037831,2015-03-25,2015-03-25,2015,0,551,S134XX,1,1,0,96.65,96.65,9,Sprain of ligaments of cervical spine,MEDICAL BACK PROBLEMS WITH MCC
1,-10000010254653,-10000930038030,2015-09-24,2015-09-24,2015,0,951,Z3480,1,1,0,6311.88,6311.88,4,Encounter for supervision of other normal pre...,OTHER FACTORS INFLUENCING HEALTH STATUS
2,-10000010254653,-10000930038031,2017-05-09,2017-05-10,2017,1,923,T7432X,1,3,0,8545.72,8545.72,3,"Child psychological abuse, confirmed","OTHER INJURY, POISONING AND TOXIC EFFECT DIAGN..."
3,-10000010254656,-10000930038162,2017-01-14,2017-01-14,2017,0,564,S8290X,1,1,0,1014.85,1014.85,4,Unspecified fracture of unspecified lower leg,OTHER MUSCULOSKELETAL SYSTEM AND CONNECTIVE TI...
4,-10000010254656,-10000930038163,2018-03-17,2018-03-17,2018,0,951,Z3480,1,1,0,9911.41,9911.41,6,Encounter for supervision of other normal pre...,OTHER FACTORS INFLUENCING HEALTH STATUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20862,-10000010288008,-10000931485965,2020-01-08,2020-01-14,2020,6,000,T50901A,1,3,0,166.97,166.97,12,"Poisoning by unspecified drugs, medicaments an...",NaN
20863,-10000010288008,-10000931485967,2020-10-23,2020-10-23,2020,0,000,T50901A,1,1,0,166.97,166.97,12,"Poisoning by unspecified drugs, medicaments an...",NaN
20864,-10000010288008,-10000931485969,2021-08-22,2021-08-23,2021,1,000,T50901A,1,1,1,166.97,166.97,13,"Poisoning by unspecified drugs, medicaments an...",NaN
20865,-10000010288008,-10000931485971,2021-09-13,2021-09-16,2021,3,000,T50901A,1,2,0,166.97,166.97,13,"Poisoning by unspecified drugs, medicaments an...",NaN


In [34]:
# prompt: Save diagnosis and inpatient_encounters as csvs

# Save diagnosis as a CSV
diagnosis.to_csv("../data/diagnosis.csv", index=False)

# Save inpatient_encounters as a CSV
inpatient_encounters.to_csv("../data/inpatient_encounters.csv", index=False)


In [35]:
inpatient_encounters['YR'].unique()

array([2015, 2017, 2018, 2022, 2021, 2020, 2023, 2016, 2019])

In [44]:
diagnosis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176690 entries, 0 to 176689
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   BENE_ID          176690 non-null  object
 1   YR               176690 non-null  int64 
 2   ICD_DIAG_CD      176690 non-null  object
 3   ICD_Description  163357 non-null  object
dtypes: int64(1), object(3)
memory usage: 6.7+ MB
